## PHYS 64 Final Project - Simulating Quantum Computers with Error

In [7]:
import numpy as np
from qiskit import QuantumCircuit

# create a quantum circuit on 3 qubits
qc = QuantumCircuit(3)

class MyQuantumCircuit(QuantumCircuit):
    def __init__(self, n):
        super().__init__(n)
        
    def x(self, q, error = 0, seed = 0):
        np.random.seed(seed)
        error = np.random.normal(0, error)
        # apply x gate to qubit q
        super().x(q)
        
        # add an error gate that doesn't show up in the circuit diagram
        if error > 0:
            super().u(error, 0, 0, q)
        
        return self
    
    def z(self, q, error = 0, seed = 0):
        np.random.seed(seed)
        error = np.random.normal(0, error)
        # apply z gate to qubit q
        super().z(q)
        
        # add an error gate that doesn't show up in the circuit diagram
        if error > 0:
            super().u(0, 0, error, q)
        
        return self
    
    def h(self, q, error = 0, seed = 0):
        np.random.seed(seed)
        error = np.random.normal(0, error)
        # apply h gate to qubit q
        super().h(q)
        
        # add an error gate that doesn't show up in the circuit diagram
        if error > 0:
            super().u(error, error, error, q)
        
        return self

In [8]:
qc = MyQuantumCircuit(3)
qc.h(0, 0.1)
qc.h(1, 0.1)
qc.h(2, 0.1)
qc.draw(output='mpl')

from qiskit.quantum_info import Statevector

def get_state(qc):
    return Statevector.from_instruction(qc).data

def get_amplitudes(qc):
    return np.abs(get_state(qc))**2

def states(qc):
    # get a list of basis states
    n = qc.num_qubits
    states = [format(i, '0'+str(n)+'b') for i in range(2**n)]
    # make states-statevector dictionary
    sv = np.round(get_state(qc), 3)
    states_dict = dict(zip(states, sv))
    return states_dict

In [1]:
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit

In [2]:
def error_correction(qc, x_ancillas, z_ancillas, logical_qubit, x_syndrome, z_syndrome):
    # Make sure inputs are valid
    if not isinstance(qc, QuantumCircuit):
        raise TypeError("'qc', should be a QuantumCircuit")
    
    if not isinstance(x_ancillas, QuantumRegister):
        raise TypeError("'x_ancillas' should be a QuantumRegister")
    elif x_ancillas.size != 3:
        raise TypeError("'x_ancillas' must have 3 qubits")
        
    if not isinstance(z_ancillas, QuantumRegister):
        raise TypeError("'z_ancillas' should be a QuantumRegister")
    elif z_ancillas.size != 3:
        raise TypeError("'z_ancillas' must have 3 qubits")
    
    if not isinstance(logical_qubit, QuantumRegister):
        raise TypeError("'logical_qubit' should be a QuantumRegister")
    elif logical_qubit.size != 7:
        raise TypeError("'logical_qubit' must have 7 qubits")
    
    if not isinstance(x_syndrome, ClassicalRegister):
        raise TypeError("'x_syndrome' should be a ClassicalRegister")
    elif x_syndrome.size != 3:
        raise TypeError("'x_syndrome' must have 3 bits")
        
    if not isinstance(z_syndrome, ClassicalRegister):
        raise TypeError("'z_syndrome' should be a ClassicalRegister")
    elif z_syndrome.size != 3:
        raise TypeError("'z_syndrome' must have 3 bits")
        
    # Initialize the ancillas to |0>
    for i in range(3):
        qc.initialize([1,0], x_ancillas[i])
        qc.initialize([1,0], z_ancillas[i])
        
    # Apply Hadamard to the ancillas
    qc.h(x_ancillas)
    qc.h(z_ancillas)
    
    # Controlled g_i
    qc.cx(z_ancillas[2], [logical_qubit[i-1] for i in [4,5,6,7]]) # Controlled g1
    qc.cx(z_ancillas[1], [logical_qubit[i-1] for i in [2,3,6,7]]) # Controlled g2
    qc.cx(z_ancillas[0], [logical_qubit[i-1] for i in [1,3,5,7]]) # Controlled g3
    
    qc.cz(x_ancillas[2], [logical_qubit[i-1] for i in [4,5,6,7]]) # Controlled g4
    qc.cz(x_ancillas[1], [logical_qubit[i-1] for i in [2,3,6,7]]) # Controlled g5
    qc.cz(x_ancillas[0], [logical_qubit[i-1] for i in [1,3,5,7]]) # Controlled g6
    
    # Apply Hadamard to the ancillas
    qc.h(x_ancillas)
    qc.h(z_ancillas)
    
    # Measure the ancillas
    qc.measure(x_ancillas, x_syndrome)
    qc.measure(z_ancillas,z_syndrome)
    
    # Apply the corrective X gates
    for i in range(1,8):
        qc.x(logical_qubit[i-1]).c_if(x_syndrome,i)
    
    # Apply the corrective Z gates
    for i in range(1,8):
        qc.z(logical_qubit[i-1]).c_if(z_syndrome,i)

In [13]:
lq = QuantumRegister(7)
x_anc  = QuantumRegister(3)
x_syn  = ClassicalRegister(3)
z_anc  = QuantumRegister(3)
z_syn  = ClassicalRegister(3)

In [14]:
qc = QuantumCircuit(x_anc,z_anc,lq,x_syn,z_syn)
error_correction(qc, x_anc, z_anc, lq, x_syn, z_syn)

In [21]:
from qiskit import execute
result = execute(qc, sv_sim).result()
sv = result.get_statevector()

ImportError: cannot import name 'execute' from 'qiskit' (/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/qiskit/__init__.py)